# HeteroJ

In [18]:
from ase.io import read
from ase.visualize import view
from pymatgen.core import Structure
from pymatgen.io.vasp.outputs import Poscar
from pymatgen.io.cif import CifWriter
import numpy as np

class HeteroJ:
    """
    用于搭建异质结结构的 python 类。
    """
    def __init__(self, file1, file2, supercell1=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], supercell2=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], 
                 mismatchA=0.001, mismatchL=0.05, vacuum=15, distance=3, **kwargs):
        self.file1 = file1
        self.file2 = file2
        self.supercell1 = supercell1
        self.supercell2 = supercell2
        self.mismatchA = mismatchA
        self.mismatchL = mismatchL
        self.vacuum = vacuum
        self.distance = distance

    def readstruct(self, file, supercell=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]):
        poscar1 = Poscar.from_file(file) 
        structure = poscar1.structure 
        structure.make_supercell(supercell)
        strucPara = str(structure.get_sorted_structure()).split('\n')
        return strucPara[2], strucPara[3]

    @staticmethod
    def make_clean(abc):
        abc_clean = abc.strip().split(" ")
        while '' in abc_clean:
            abc_clean.remove('')
        return list(map(lambda x: float(x), abc_clean[-3:]))

    def makeJunction(self):
        poscar1 = Poscar.from_file(self.file1)
        structure1 = poscar1.structure
        structure1.make_supercell(self.supercell1)
        
        poscar2 = Poscar.from_file(self.file2)
        structure2 = poscar2.structure
        structure2.make_supercell(self.supercell2)
    
        zlist1 = []
        for i in range(len(structure1.species)):
            zlist1.append(float(structure1.frac_coords[i][-1]))
        zmax1,zmin1 = max(zlist1), min(zlist1)
        thick1 = zmax1-zmin1
        zlist2 = []
        for i in range(len(structure2.species)):
            zlist2.append(float(structure2.frac_coords[i][-1]))
        zmax2,zmin2 = max(zlist2), min(zlist2)
        thick2 = zmax2 - zmin2

        # 使用 file1 的晶格参数
        abc1, angle1 = self.readstruct(self.file1, self.supercell1)
        abc1_clean = make_clean(abc1)
    
        thicktot = (thick1 + thick2)*float(abc1_clean[-1]) + self.distance + self.vacuum
    
        for i in range(len(structure2.species)):
            newcoordlist = []
            newcoordlist.append(structure2.frac_coords[i][0])
            newcoordlist.append(structure2.frac_coords[i][1])
            newcoordlist.append(structure2.frac_coords[i][-1] + zmax1 - zmin2 + self.distance/float(abc1_clean[-1]))
            structure1.append(species=structure2.species[i], coords=newcoordlist, coords_are_cartesian=False)

        open("hetero.vasp", "w").write(str(Poscar(structure1).get_string(direct=False)))
        f = open("hetero.vasp", 'r', encoding='utf-8')
        f_new = open("hetero_with_vaccum.vasp", 'w', encoding='utf-8')
        count = 0
        for line in f:
            count += 1
            if count == 5:
                line = "0.0 0.0 " + str(thicktot) + "\n"
                f_new.write(line)
                continue
            f_new.write(line)
        f.close()
        f_new.close()

    def build(self):
        abc1, angle1 = self.readstruct(self.file1, self.supercell1)
        abc2, angle2 = self.readstruct(self.file2, self.supercell2)
        
        abc1_clean = make_clean(abc1)
        angle1_clean = make_clean(angle1)
        abc2_clean = make_clean(abc2)
        angle2_clean = make_clean(angle2)

        mismatch_angle3 = abs(angle1_clean[2] - angle2_clean[2]) / min((angle1_clean[2], angle2_clean[2]))
        
        if mismatch_angle3 < self.mismatchA:
            pass
        else:
            judgeSmallest = 100
            for a in range(-2, 3):
                for b in range(-2, 3):
                    for i in range(-2, 3):
                        for j in range(-2, 3):
                            if a == 0 or i == 0:
                                continue
                            abc1, angle1 = self.readstruct(self.file1, [[a, b, 0],[0, 1, 0],[0, 0, 1]])
                            abc2, angle2 = self.readstruct(self.file2, [[i, j, 0],[0, 1, 0],[0, 0, 1]])
                            abc1_clean = make_clean(abc1)
                            angle1_clean = make_clean(angle1)
                            abc2_clean = make_clean(abc2)
                            angle2_clean = make_clean(angle2)
                            mismatch_angle3 = abs(angle1_clean[2] - angle2_clean[2]) / min((angle1_clean[2], angle2_clean[2]))
                            if mismatch_angle3 < mismatchA:
                                if abs(a) + abs(b) + abs(i) + abs(j) >= judgeSmallest:
                                    continue
                                judgeSmallest = abs(a) + abs(b) + abs(i) + abs(j)
                                self.supercell1 = [[a, b, 0],[0, 1, 0],[0, 0, 1]]
                                self.supercell2 = [[i, j, 0],[0, 1, 0],[0, 0, 1]]
                                print("\nmismatch =", mismatch_angle3)
                                print("Find a new supercellgroup:", self.supercell1, self.supercell2)
                                print(abc1, angle1)
                                print(abc2, angle2)
                                break
                            else:
                                continue
        
        abc1, angle1 = self.readstruct(self.file1, self.supercell1)
        abc2, angle2 = self.readstruct(self.file2, self.supercell2)
        abc1_clean = make_clean(abc1)
        angle1_clean = make_clean(angle1)
        abc2_clean = make_clean(abc2)
        angle2_clean = make_clean(angle2)
        
        
        mismatch_abc1 = abs(abc1_clean[0] - abc2_clean[0]) / min(abs(abc2_clean[0]), abs(abc2_clean[0]))
        mismatch_abc2 = abs(abc1_clean[1] - abc2_clean[1]) / min(abs(abc2_clean[1]), abs(abc2_clean[1]))
        
        if mismatch_abc1 < self.mismatchL and mismatch_abc2 < self.mismatchL:
            pass
        else:
            judgeSmallest = 100
            for a in range(1, 9):
                for b in range(1, 9):
                    for i in range(1, 9):
                        for j in range(1, 9):
                            mismatch_abc1 = abs(abc1_clean[0] * a-abc2_clean[0] * i) / min(abs(abc2_clean[0]) * a, abs(abc2_clean[0] * i))
                            mismatch_abc2 = abs(abc1_clean[1] * b-abc2_clean[1] * j) / min(abs(abc2_clean[1]) * b, abs(abc2_clean[1] * j))
                            if mismatch_abc1 < self.mismatchL and mismatch_abc2 < self.mismatchL:
                                if abs(a) + abs(b) + abs(i) + abs(j) >= judgeSmallest:
                                    continue
                                judgeSmallest = abs(a) + abs(b) + abs(i) + abs(j)
                                Lengthsuperlist = [a, b, i, j]
            if 'Lengthsuperlist' not in dir():
                print("@@@@@@@@@ reverse one of the vector and retry @@@@@@@@@@")
                judgeSmallest = 100
                for a in range(1, 9):
                    for b in range(1, 9):
                        for i in range(1, 9):
                            for j in range(1, 9):
                                mismatch_abc1 = abs(abc1_clean[1] * a - abc2_clean[0] * i) / min(abs(abc2_clean[1]) * a, abs(abc2_clean[0] * i))
                                mismatch_abc2 = abs(abc1_clean[0] * b - abc2_clean[1] * j) / min(abs(abc2_clean[0]) * b, abs(abc2_clean[1] * j))
                                if mismatch_abc1 < self.mismatchL and mismatch_abc2 < self.mismatchL:
                                    if abs(a) + abs(b) + abs(i) + abs(j) >= judgeSmallest:
                                        continue
                                    judgeSmallest = abs(a) + abs(b) + abs(i) + abs(j)
                                    Lengthsuperlist = [b, a, i, j]
                                    print(Lengthsuperlist)
        
            self.supercell1[0][0] = self.supercell1[0][0] * Lengthsuperlist[0]
            self.supercell1[0][1] = self.supercell1[0][1] * Lengthsuperlist[0]
            self.supercell1[1][0] = self.supercell1[1][0] * Lengthsuperlist[1]
            self.supercell1[1][1] = self.supercell1[1][1] * Lengthsuperlist[1]
            self.supercell2[0][0] = self.supercell2[0][0] * Lengthsuperlist[2]
            self.supercell2[0][1] = self.supercell2[0][1] * Lengthsuperlist[2]
            self.supercell2[1][0] = self.supercell2[1][0] * Lengthsuperlist[3]
            self.supercell2[1][1] = self.supercell2[1][1] * Lengthsuperlist[3]
        
        abc1, angle1 = self.readstruct(self.file1, self.supercell1)
        abc2, angle2 = readstruct(self.file2, self.supercell2)
        abc1_clean = make_clean(abc1)
        angle1_clean = make_clean(angle1)
        abc2_clean = make_clean(abc2)
        angle2_clean = make_clean(angle2)

        self.makeJunction()
        
        # print("############## Summary ################")
        # print("The final supercell for ", self.file1)
        # print(self.supercell1[0], "\n", self.supercell1[1], "\n", self.supercell1[2], "\n")
        # print("The final supercell for ", self.file2)
        # print(self.supercell2[0], "\n", self.supercell2[1], "\n", self.supercell2[2], "\n")
        # print("Final mismatch of Angel:", abs(angle1_clean[2] - angle2_clean[2]) / min((angle1_clean[2], angle2_clean[2])))
        # mismatch_abc1 = abs(abc1_clean[0] - abc2_clean[0]) / min(abs(abc2_clean[0]), abs(abc2_clean[0]))
        # mismatch_abc2 = abs(abc1_clean[1] - abc2_clean[1]) / min(abs(abc2_clean[1]), abs(abc2_clean[1]))
        # print("Final mismatch of Length a and b:", mismatch_abc1, " ", mismatch_abc2)
        # print("Use ", self.file1, " as substrate\n"
        #       "Distance between two slab is", self.distance, "Angstrom"
        #       "\nVacuum between two slab is", self.vacuum, "Angstrom")
        # print("############## Summary ################")

        atoms = read("hetero_with_vaccum.vasp")
        atoms.center()
        return atoms

In [22]:
system = HeteroJ(file1="../examples/Heterostructure/03_black_P_POSCAR", file2="../examples/Heterostructure/03_Graphene_POSCAR")
atoms = system.build()
view(atoms)
# write("mbene.vasp", atoms, direct=True, sort=True, vasp5=True)


mismatch = 0.0
Find a new supercellgroup: [[-2, 0, 0], [0, 1, 0], [0, 0, 1]] [[-2, -1, 0], [0, 1, 0], [0, 0, 1]]
abc   :   8.760000   3.310000  17.068001 angles:  90.000000  90.000000  90.000000
abc   :   4.274223   2.467724  15.000000 angles:  90.000000  90.000000  90.000000

mismatch = 0.0
Find a new supercellgroup: [[-1, 0, 0], [0, 1, 0], [0, 0, 1]] [[-2, -1, 0], [0, 1, 0], [0, 0, 1]]
abc   :   4.380000   3.310000  17.068001 angles:  90.000000  90.000000  90.000000
abc   :   4.274223   2.467724  15.000000 angles:  90.000000  90.000000  90.000000


<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\Q-Ap...>